# Build a simple ML pipeline with spark component

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription - [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace with computer cluster - [Configure workspace](../../configuration.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](../../../README.md) - check the getting started section

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Connect to your AML workspace from the Python SDK
- Create `Pipeline` with spark component

**Motivations** - In this example, we will explains how to create a spark component and use it in a pipeline. A Spark Component is a Component that executes a spark job in AML. It will support attached synapse spark and hobo spark.

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.2 Prepare spark workspace and compute resource
1. **Create an Azure Synapse workspace**, check [this](https://docs.microsoft.com/en-us/azure/synapse-analytics/quickstart-create-workspace) for more information.
2. **Create compute resource**, you can select from following two options:

    - Submit a Spark Job using HOBO compute (cluster-less or serverless), check [this](https://docs.microsoft.com/en-us/azure/synapse-analytics/quickstart-create-apache-spark-pool-portal) for more information about creating Spark Pool in Synapse workspace.
    
    - Submit a Spark Job using an attached Synapse compute, check [this](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-link-synapse-ml-workspaces) for more information about linking Azure Synapse Analytics and Azure Machine Learning workspaces, and attach Apache Spark pools.

    In this example, we have created synapse spark compute in our CI workspace ("spark31").

## 1.1 Import the required libraries

In [1]:
# import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient, Input, Output, load_component, command
from azure.ai.ml.dsl import pipeline
from azure.ai.ml.entities import Environment
from azure.ai.ml.constants import AssetTypes, InputOutputModes

## 1.2 Configure credential
We are using `DefaultAzureCredential` to get access to workspace.

`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 

Reference for more available credentials if it does not work for you: [configure credential example](../../configuration.ipynb), [azure-identity reference doc](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [2]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

## 1.3 Get a handle to the workspace

We use config file to connect to a workspace. The Azure ML workspace should be configured with computer cluster. [Check this notebook for configure a workspace](../../configuration.ipynb)

In [4]:
# Get a handle to workspace
#ml_client = MLClient.from_config(credential=credential)
sub_id = "b746917e-ceb7-4ae0-81e6-3ccd893cb0de"
rg = "dpv2"
workspace = "dpv2-wks"

ml_client = MLClient(DefaultAzureCredential(), sub_id, rg, workspace)

# Retrieve an already attached Azure Machine Learning Synapse Compute.
#spark_compute_target = "spark31"
spark_compute_target = "spark-cluster"
print(ml_client.compute.get(spark_compute_target))

# Retrieve an already attached Azure Machine Learning Compute.
cluster_name = "cpu-cluster"
print(ml_client.compute.get(cluster_name))

SynapseSparkCompute({'type': 'synapsespark', 'created_on': datetime.datetime(2022, 10, 6, 6, 18, 34, 533622, tzinfo=<FixedOffset '+00:00'>), 'provisioning_state': 'Succeeded', 'provisioning_errors': None, 'name': 'spark-cluster', 'description': None, 'tags': {}, 'properties': {}, 'id': '/subscriptions/b746917e-ceb7-4ae0-81e6-3ccd893cb0de/resourceGroups/dpv2/providers/Microsoft.MachineLearningServices/workspaces/dpv2-wks/computes/spark-cluster', 'Resource__source_path': None, 'base_path': 'd:\\code\\dpv2demo\\Spark', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x000001CA89A278B0>, 'resource_id': '/subscriptions/b746917e-ceb7-4ae0-81e6-3ccd893cb0de/resourceGroups/rg_synapse/providers/Microsoft.Synapse/workspaces/synapse-wks/bigDataPools/sparkpool', 'location': 'eastus', 'identity': None, 'node_count': 0, 'node_family': 'MemoryOptimized', 'node_size': 'Small', 'spark_version': '3.2', 'scale_settings': <azure.ai.ml.entities._compute.synapsespark_comput

# 2. Define components

Use `load_component` to load spark components defined using YAML. 

In [5]:
# load component
parent_dir = "."
spark_kmeans = load_component(path=parent_dir + "/components/spark_kmeans.yml")

show_output_component = command(
    inputs=dict(spark_output=Input(type=AssetTypes.URI_FOLDER)),
    command="ls ${{inputs.spark_output}}",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu:1",
)

# 3. Build pipeline

We define a pipeline containing 2 nodes:
- `kmeans_cluster` is a spark component which will conduct kmeans algorithom and print results.
- `show_output` is a command component which will show center points got from kmeans_cluster

In [6]:
@pipeline()
def spark_pipeline_from_yaml(train_data):
    kmeans_clustering = spark_kmeans(file_input=train_data)
    kmeans_clustering.compute = spark_compute_target
    kmeans_clustering.outputs.output.mode = InputOutputModes.DIRECT

    show_output = show_output_component(spark_output=kmeans_clustering.outputs.output)


sample_data = Input(
    path=parent_dir + "/data/sample_kmeans_data.txt",
    type=AssetTypes.URI_FOLDER,
    mode=InputOutputModes.DIRECT,
)

pipeline_job = spark_pipeline_from_yaml(train_data=sample_data)

# set pipeline level compute
pipeline_job.settings.default_compute = cluster_name

In [7]:
print(pipeline_job)

display_name: spark_pipeline_from_yaml
type: pipeline
inputs:
  train_data:
    mode: direct
    type: uri_folder
    path: azureml:./data/sample_kmeans_data.txt
outputs: {}
jobs:
  kmeans_clustering:
    $schema: '{}'
    type: spark
    inputs:
      file_input:
        path: ${{parent.inputs.train_data}}
    outputs:
      output:
        mode: direct
        type: uri_folder
    code: D:/code/dpv2demo/Spark/components
    entry:
      file: kmeans_example.py
    args: --file_input ${{inputs.file_input}} --output ${{outputs.output}}
    conf:
      spark.yarn.appMasterEnv.AZUREML_HADOOP_EXTENSION_URL: https://foobaradrama2.azurefd.net/latest/hadoop-azureml-fs.jar
      spark.yarn.appMasterEnv.AZUREML_ENABLE_DATAPATH_RESOLUTION: true
      spark.driver.cores: 2
      spark.driver.memory: 1g
      spark.executor.cores: 1
      spark.executor.memory: 1g
      spark.executor.instances: 1
    component:
      $schema: http://azureml/sdk-2-0/SparkComponent.json
      name: kmeans_spark_co

# 4. Submit pipeline job

In [8]:
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="pipeline_samples"
)
pipeline_job

Experiment,Name,Type,Status,Details Page
pipeline_samples,clever_bottle_nj584yxc8r,pipeline,Preparing,Link to Azure Machine Learning studio


In [9]:
# wait until the job completes
ml_client.jobs.stream(pipeline_job.name)

RunId: clever_bottle_nj584yxc8r
Web View: https://ml.azure.com/runs/clever_bottle_nj584yxc8r?wsid=/subscriptions/b746917e-ceb7-4ae0-81e6-3ccd893cb0de/resourcegroups/dpv2/workspaces/dpv2-wks

Streaming logs/azureml/executionlogs.txt

[2022-10-06 06:46:47Z] Submitting 1 runs, first five are: b947448d:aca37313-b8b3-4555-942e-5b59d7f857b1
[2022-10-06 06:48:46Z] Execution of experiment failed, update experiment status and cancel running nodes.

Execution Summary
RunId: clever_bottle_nj584yxc8r
Web View: https://ml.azure.com/runs/clever_bottle_nj584yxc8r?wsid=/subscriptions/b746917e-ceb7-4ae0-81e6-3ccd893cb0de/resourcegroups/dpv2/workspaces/dpv2-wks


JobException: Exception : 
 {
    "error": {
        "code": "UserError",
        "message": "Pipeline has some failed steps. See child run or execution logs for more details.",
        "message_format": "Pipeline has some failed steps. {0}",
        "message_parameters": {},
        "reference_code": "PipelineHasStepJobFailed",
        "details": []
    },
    "environment": "eastus",
    "location": "eastus",
    "time": "2022-10-06T06:48:46.726874Z",
    "component_name": ""
} 

# Next Steps
You can see further examples of running a pipeline job [here](../)